In [ ]:

!pip install helical

!pip install datasets --upgrade

from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import anndata as ad
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from scipy.sparse import lil_matrix


est


In [ ]:
from datasets import load_dataset
ds = load_dataset("helical-ai/yolksac_human",trust_remote_code=True, split="train[:65%]",download_mode="reuse_cache_if_exists")

In [ ]:
observation_columns = [obs for obs in list(ds.features.keys()) if not obs == 'raw_counts']
obs_data = pd.DataFrame(ds.select_columns(observation_columns).data.to_pandas(),columns=observation_columns)
lil = lil_matrix((len(ds),ds[0]['size']))
lil.data = np.array(ds['raw_counts'],dtype="object")
lil.rows = np.array(ds['rows'],dtype="object")
adata = ad.AnnData(lil.tocsr(),obs=obs_data)
adata.var_names = ds.features['raw_counts'].id.split(",")
adata.var['gene_name'] = adata.var_names.str.upper()

In [ ]:
# get labels: the celltype
num_types = adata.obs["LVL1"].unique().shape[0]
id2type = dict(enumerate(adata.obs["LVL1"].astype("category").cat.categories))

celltypes_labels = np.array(adata.obs["LVL1"].tolist())

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
scgpt_config = scGPTConfig(batch_size=50, device=device)
scgpt = scGPT(configurer = scgpt_config)
print(adata.var.index)


def knockout_genes(data,model,genes):
  """ Generate embeddings for the original dataset and modified dataset using
  the selected model. The modified dataset has gene expressions set to 0 for
  all genes present in the genes list.

  Inputs:
      data (anndata): dataset that is being considered
      model ()
  """
  # knockout genes using batching to reduce explosion in RAM usage
  batch_size = 200
  x_model_dropped_list = []
  for i in range(0, len(genes), batch_size):
        batch_genes = genes[i:i+batch_size]
        # set expression values for genes in the current batch to 0
        for gene in batch_genes:
            if gene in data.var.index:
                gene_idx = list(data.var.index).index(gene)
                data.X[:, gene_idx] = 0

        # generate embeddings for the current batch of knockouts
        new_data = model.process_data(adata=data, gene_names="gene_name")
        x_model_dropped = model.get_embeddings(new_data)
        x_model_dropped_list.append(x_model_dropped)
  # generate embeddings on original data
  new_data = model.process_data(adata = data, gene_names = "gene_name")
  x_model = model.get_embeddings(new_data)
  print(x_model, x_model_dropped)
  return x_model, x_model_dropped
#genes_to_knockout = ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M']
genes_to_knockout = list(adata.var_names[0:int(len(adata.var_names)/2)])

x_model, x_model_dropped = knockout_genes(adata,scgpt,genes_to_knockout)
